### scrape hub

In [2]:
#from urllib.request import urlopen, Request, urlretrieve
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re

DRIVER_PATH = '/usr/sbin/chromedriver'
URLS = "./sorted.txt"
OUTPUT = "./output"
FAILED = "./failure.txt"

In [9]:
pwd

'/home/peat/hack/scwape/scihub'

In [11]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
               "download.default_directory": OUTPUT ,
               "download.directory_upgrade": True,
               "plugins.always_open_pdf_externally": True,
               "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.implicitly_wait(0.5)
pdf= re.compile('^location\.href=\'(.*)\'$')
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

/tmp/ipykernel_9740/559492227.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


In [4]:
pdffound = []
cantdoit = []
for url in open(URLS, "r"):
    yurl = 'https://sci-hub.ru/'+url
    driver.get(yurl)
    try:
        button = driver.find_element(By.TAG_NAME, "button")
        #print(dir(button))
        #print(button.get_attribute('onclick'))
        pdfloc = pdf.search(button.get_attribute('onclick'))
        title = driver.find_element(By.TAG_NAME, "title")
        #print(dir(title))
        #print(title.get_attribute("innerHTML"))
        #print(pdfloc.group(1))
        pdffound.append((title.get_attribute("innerHTML"), pdfloc.group(1)))
    except:
        cantdoit.append(yurl)
        

In [13]:
for (title, pdfname) in pdffound:
        yurl = "https://sci-hub.ru"+pdfname
        driver.get(yurl)

In [14]:
with open(FAILED, "w") as failed:
    for failure in cantdoit:
        failed.write(failure+"\n")